<a href="https://colab.research.google.com/github/lkjhgfdbf3/AI-15-Section1-Project/blob/main/Section4_Project_MobileNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip -qq "/content/drive/MyDrive/01.데이터.zip"

In [ ]:
!pip install split-folders

In [ ]:
import splitfolders

In [ ]:
splitfolders.ratio('/content/01.데이터', output="output", seed=1337, ratio=(0.7, 0.2, 0.1))

In [ ]:
import numpy as np

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model

mobilenet = MobileNet(weights='imagenet', include_top=False)

In [ ]:
import os

base_dir = '/content/output'

train_dir = os.path.join(base_dir, 'train')
train_normal_dir = os.path.join(train_dir, '무')
train_disease_dir = os.path.join(train_dir, '유')

validation_dir = os.path.join(base_dir, 'val')
validation_normal_dir = os.path.join(validation_dir, '무')
validation_disease_dir = os.path.join(validation_dir, '유')

In [ ]:
print('total training normal images:', len(os.listdir(train_normal_dir)))
print('total training disease images:', len(os.listdir(train_disease_dir)))
print('total validation normal images:', len(os.listdir(validation_normal_dir)))
print('total validation disease images:', len(os.listdir(validation_disease_dir)))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,  
        target_size=(224, 224),
        batch_size=20,
        #classes=['nor','dis'],
        class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=20,
        #classes=['nor','dis'],
        class_mode='binary')

In [ ]:
import matplotlib.pylab as plt

#class_labels = ['nor', 'dis']
batch = next(train_generator)
images, labels = batch[0], batch[1] # 0번 이미지데이터 1번 레이블
print(labels[:5])
plt.figure(figsize=(16,8))
for i in range(5) :
    ax = plt.subplot(4,8,i+1)
    plt.imshow(images[i])
    #plt.title(class_labels[labels[i].astype(np.int)])
    plt.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
from tensorflow.keras import layers
img_input = layers.Input(shape=(224*224*3))

In [ ]:
for layer in mobilenet.layers:
    layer.trainable = False

In [ ]:
x = mobilenet.output
x = GlobalAveragePooling2D()(x)
#x = Dense(512, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(mobilenet.input, predictions)

In [ ]:
from tensorflow import keras

checkpoint_filepath = "FMbest.hdf5"
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=0,patience=5, verbose=1)

save_best = keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_loss',
                                            verbose=1, save_best_only=True, save_weights_only=True, 
                                            mode='auto',save_freq='epoch',options=None)

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=27,  
      epochs=20,
      validation_data=validation_generator,
      validation_steps=10,
      callbacks=[early_stop, save_best],
      verbose=1)

In [ ]:
plt.plot(history.history['accuracy'],'b-', label='accuracy')
plt.plot(history.history['val_accuracy'],'r--', label='val_accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'],'b-', label='loss')
plt.plot(history.history['val_loss'],'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
test_dir = os.path.join(base_dir, 'test')
test_normal_dir = os.path.join(test_dir, '무')
test_disease_dir = os.path.join(test_dir, '유')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=20,
        #classes=['nor','dis'],
        class_mode='binary')

In [ ]:
model.evaluate(test_generator)